# 삼성 DS-KAIST AI Expert 프로그램
## 실습 평가: Visual Question Answering

담당 조교: 정종헌 (jongheonj@kaist.ac.kr), 박종진 (jongjin.park@kaist.ac.kr)

### 필요 환경
- python==3.7
- tensorflow==1.14
- numpy

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# 과제 실행에 필요한 라이브러리를 로드합니다.
import tensorflow as tf

# TensorFlow에서 만들어진 computational graph를 실행할 수 있는 Session을 생성합니다.
sess = tf.InteractiveSession()

# 본 과제에서 사용할 견본 feature map입니다.
# 32개의 batch로 구성된 24개의 128x128 feature maps로 생각할 수 있습니다.
# 해당 `base_features` 변수를 중간에 변경하지 않도록 주의하시길 바랍니다.
base_features = tf.zeros((32, 128, 128, 24))

# `base_features` 변수를 연산하고 출력합니다.
print(sess.run(base_features))

### **개요**

실습 과정에서 다루었던 [Relational Network](https://arxiv.org/abs/1706.01427)에서는 다음과 같은 형태의 뉴럴 아키텍쳐 $f_\phi$ 및 $g_\theta$를 고려합니다: 

$$a = f_\phi\left(\sum_{i, j} g_\theta (o_i, o_j, q) \right)$$

해당 아키텍쳐의 내부 구현에서는 **positional encoding**을 수행하는 과정이 있습니다. 이는 주어진 feature maps의 pixel 위치 정보에 무관하게 동작하는 relation model ($g_\theta$)에 각 픽셀의 위치 정보를 제공하기 위한 목적으로, 주어진 features $o$의 각 픽셀 ($o_i$) 마다 정규화된 좌표의 위치 $(x, y)$를 새로운 feature로 concatenate하는 방식으로 구현됩니다. 즉, 주어진 $B\times D \times D \times C$ 형태의 feature $o$에 대해, 아래 `positional_encoding()` 함수는 $B\times D \times D \times (C+2)$ 형태의 새로운 feature maps를 반환합니다:

In [ ]:
def positional_encoding(features):
    # Append two features of positional encoding to the given feature maps
    d = features.get_shape().as_list()[1]
    indices = tf.range(d)
    x = tf.tile(tf.reshape(indices, [d, 1]), [1, d])
    y = tf.tile(tf.reshape(indices, [1, d]), [d, 1])
    pos = tf.cast(tf.stack([x, y], axis=2)[None] / d, tf.float32)
    pos = tf.tile(pos, [tf.shape(features)[0], 1, 1, 1])
    return tf.concat([features, pos], axis=3)

pe_features = positional_encoding(base_features)
print(f"Original feature: {base_features.shape}")
print(f"PE-ed feature: {pe_features.shape}")

In [ ]:
# positional encoding을 수행한 변수를 연산하고 출력합니다.
print(sess.run(pe_features))

### **과제 설명**

하지만, positional encoding을 구현하는 방식이 유일한 것은 아닙니다. 본 과제의 목표는 아래 조건을 충족하는 **새로운 positinal encoding 함수 `positional_encoding_angular()`를 구현**하는 것입니다:
- 입력형태와 출력형태는 $B\times D \times D \times C$ 로 같습니다.
- 기존 `positional_encoding()` 함수가 2-dimensional 채널을 구성하여 기존 features에 concatenate했던 것과는 달리, `positional_encoding_angular()`는 주어진 입력 $o$와 **같은 형태**의 tensor를 만들어 원래 tensor에 **더해주는 방식**을 가집니다. 수학적으로, 특정 pixel의 object $o_\mathrm{pos}\in\mathbb{R}^{C}$에 대해 목표 함수를 다음과 같이 기술합니다:

$$PE(o_\mathrm{pos}) = o_{\mathrm{pos}} + Encode(\mathrm{pos}, C) \text{, where } Encode(\mathrm{pos}, C) \in \mathbb{R}^{C}$$

- 즉, tensor의 형태로는 다음과 같이 쓸 수 있습니다: $PE(o) = o + Encode(:, C) \in \mathbb{R}^{D\times D \times C}$

- 각 pixel의 위치 $(x, y)=(0, 0)$ ~ $(D-1, D-1)$에 대해, $\mathrm{pos}:= Dx + y$ 로 정의합니다.

- 주어진 $\mathrm{pos}$와 채널의 수 $C$에 대해, $Encode_i(\mathrm{pos}, C)$는 $(i=0, \cdots, C-1)$ 다음과 같이 정의됩니다:

$$Encode_{i}(\mathrm{pos}, C) = \sin(\mathrm{pos}/(10000^{2k/C})) \text{, if }i=2k$$

$$Encode_{i}(\mathrm{pos}, C) = \cos(\mathrm{pos}/(10000^{2k/C})) \text{, if }i=2k+1$$

아래 **"MODIFY HERE"** 블록 내부를 수정하여 함수의 구현을 완성해 주세요.

In [ ]:
def positional_encoding_angular(features):
    shapes = features.shape.as_list()
    D, C = shapes[1], shapes[3]
    K = C // 2
    
    def _encode_at_k(k):
        """Compute [Encode_{2k}(:, C), Encode_{2k+1}(:, C)]
        
        Returns:
            A tensor of shape (D x D x 2)
        """
        indices = tf.range(D)
        x = tf.tile(tf.reshape(indices, [D, 1]), [1, D])
        y = tf.tile(tf.reshape(indices, [1, D]), [D, 1])

        ###################### MODIFY HERE ######################

        pos = D * x + y
        mult = 10000 ** (2*k / C)

        angle = tf.cast(pos, tf.float32) / mult
        f_sin = tf.sin(angle)
        f_cos = tf.cos(angle)

        return tf.stack([f_sin, f_cos], axis=2)

        #########################################################
    
    
    fs = [_encode_at_k(k) for k in range(K)]
    encode = tf.concat(fs, axis=2)[None]
    pe = features + encode
    
    return pe

In [ ]:
pe_angluar_features = positional_encoding_angular(base_features)
print(f"Original feature: {base_features.shape}")
print(f"PE-ed feature: {pe_angluar_features.shape}")

In [ ]:
print(sess.run(pe_angluar_features))